In [1]:
import numpy as np
import pandas as pd
import textwrap
import nltk
from nltk import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [2]:
df = pd.read_csv('bbc_text_cls.csv')
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [3]:
labels = set(df['labels'])
labels

{'business', 'entertainment', 'politics', 'sport', 'tech'}

In [4]:
# Choose a label data we want to train
label = 'business'

In [5]:
# selecting only business articles
texts = df[df['labels'] == label]['text']
texts.head()

0    Ad sales boost Time Warner profit\n\nQuarterly...
1    Dollar gains on Greenspan speech\n\nThe dollar...
2    Yukos unit buyer faces loan claim\n\nThe owner...
3    High fuel prices hit BA's profits\n\nBritish A...
4    Pernod takeover talk lifts Domecq\n\nShares in...
Name: text, dtype: object

In [6]:
# Collect counts
probs  = {} # key: (w(t-1), w(t+1)) value: {w(t): count(w(t))}

for doc in texts:
    lines = doc.split("\n")
    for line in lines:
        tokens = word_tokenize(line)
        for i in range (len(tokens) - 2):
            t_0 = tokens[i]
            t_1 = tokens[i+1]
            t_2 = tokens[i+2]
            key = (t_0, t_2)
            if key not in probs:
                probs[key] = {}
                
            # add count for middle token
            if t_1 not in probs[key]:
                probs[key][t_1] = 1
            else:
                probs[key][t_1] = 1

In [7]:
# Normalize probabilities

for key, d in probs.items():
    # d should represent a distribution
    total = sum(d.values())
    for k, v in d.items():
        d[k] = v/total

In [8]:
probs

{('Ad', 'boost'): {'sales': 1.0},
 ('sales', 'Time'): {'boost': 1.0},
 ('boost', 'Warner'): {'Time': 1.0},
 ('Time', 'profit'): {'Warner': 1.0},
 ('Quarterly', 'at'): {'profits': 1.0},
 ('profits', 'US'): {'at': 1.0},
 ('at', 'media'): {'US': 1.0},
 ('US', 'giant'): {'media': 0.125,
  'telecoms': 0.125,
  'banking': 0.125,
  'foods': 0.125,
  'retail': 0.125,
  'oil': 0.125,
  'mortgage': 0.125,
  'agrochemical': 0.125},
 ('media', 'TimeWarner'): {'giant': 1.0},
 ('giant', 'jumped'): {'TimeWarner': 1.0},
 ('TimeWarner', '76'): {'jumped': 1.0},
 ('jumped', '%'): {'76': 0.14285714285714285,
  '1.8': 0.14285714285714285,
  '11': 0.14285714285714285,
  '6': 0.14285714285714285,
  '10.7': 0.14285714285714285,
  '7': 0.14285714285714285,
  '22': 0.14285714285714285},
 ('76', 'to'): {'%': 1.0},
 ('%', '$'): {'to': 0.5, 'at': 0.5},
 ('to', '1.13bn'): {'$': 1.0},
 ('$', '('): {'1.13bn': 0.009174311926605505,
  '900m': 0.009174311926605505,
  '280bn': 0.009174311926605505,
  '86m': 0.00917431192

In [9]:
texts.iloc[0].split("\n")

['Ad sales boost Time Warner profit',
 '',
 'Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.',
 '',
 'The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.',
 '',
 "Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers a

In [10]:
def spin_document(doc):
    # SPlit the document into lines (pragraphs)
    lines = doc.split("\n")
    output = []
    for line in lines:
        if line:
            new_line = spin_line(line)
        else:
            new_line = line
        output.append(new_line)
    
    return "\n".join(output)

In [11]:
detokenizer  = TreebankWordDetokenizer()

In [12]:
texts.iloc[0].split("\n")[2]

'Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.'

In [15]:
sample_tokens = word_tokenize(texts.iloc[0].split("\n")[2])

In [16]:
detokenizer.detokenize(sample_tokens)

'Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.'

In [17]:
def sample_word(d):
    p0 = np.random.random()
    cumulative = 0
    for t, p in d.items():
        cumulative += p
        if p0< cumulative:
            return t
    assert(False)

In [18]:
def spin_line(line):
    tokens = word_tokenize(line)
    i = 0
    output = [tokens[0]]
    while i < (len(tokens)-2):
        t_0 = tokens[i]
        t_1 = tokens[i+1]
        t_2 = tokens[i+2]
        key = (t_0, t_2)
        p_dist = probs[key]
        
        if len(p_dist) > 1 and np.random.random() <0.3:
            # let's replace the middle word
            middle = sample_word(p_dist)
            output.append(t_1)
            output.append("<"+ middle + ">")
            output.append(t_2)
            
            # We won't replace the 3rd token since the middle
            # token was dependent on it
            # isntead, skip ahead 2 steps
            i += 2
        else:
            # we won't replace this middle word
            output.append(t_1)
            i += 1
     # append the final token - only if there was no replacement
    if i == len(tokens) - 2:
        output.append(tokens[-1])
    return detokenizer.detokenize(output)

In [19]:
np.random.seed(1234)


In [22]:
i = np.random.choice(texts.shape[0])
doc = texts.iloc[i]
new_doc = spin_document(doc)
print(doc)
print("\n\nThe following is New Document \n\n")
print(new_doc)

Ukraine revisits state sell-offs

Ukraine is preparing what could be a wholesale review of the privatisation of thousands of businesses by the previous administration.

The new President, Viktor Yushchenko, has said a "limited" list of companies is being drawn up. But on Wednesday Prime Minister Yulia Tymoshenko said the government was planning to renationalise 3,000 firms. The government says many privatised firms were sold to allies of the last administration at rock-bottom prices. More than 90,000 businesses in all, from massive corporations to tiny shopfronts, have been sold off since 1992, as the command economy built up when Ukraine was part of the Soviet Union was dismantled.

Ms Tymoshenko said prosecutors had drawn up a list of more than 3,000 businesses which were to be reviewed. "We will return to the state that which was illegally put into private hands."

A day earlier, Mr Yushchenko - keen to reassure potential investors - had said only 30 to 40 top firms would be targete

In [23]:
print(textwrap.fill(new_doc, replace_whitespace=False, fix_sentence_endings=True))

Ukraine revisits state sell-offs

Ukraine is preparing what could be
<cause> a wholesale review of the privatisation of thousands of
businesses by the previous <last> administration.

The new President,
Viktor Yushchenko <Yanukovich>, has <"> said a "limited" list of
companies is <are> being drawn up . But on Wednesday Prime Minister
Yulia Tymoshenko said the government was planning to renationalise
3,000 firms . The government says many privatised <privatised> firms
were sold to allies of the last <Bush> administration at rock-bottom
<rock-bottom> prices . More than 90,000 businesses in all <deficit>,
from massive corporations to tiny shopfronts, have been sold off since
1992, as the command <Mexican> economy built <built> up when Ukraine
was part of the Soviet Union was dismantled.

Ms Tymoshenko
<Tymoshenko> said prosecutors <2004> had drawn up a list <battery> of
more than 3,000 businesses which were to be reviewed . "We <I> will
return <double> to the state <shares> that which was